## 챗봇 만들기
### 데이터셋 만들기

In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
def create_df(folder):
    FILE_NAME = glob.glob(f'./data/*/{folder}/***.txt')
    dic = {'Q':[], 'A':[]}
    for file in FILE_NAME:
        with open(file, 'r', encoding = 'utf-8') as f:
            content = f.readlines()
        post = content[0][0]
        N = 0
        for i in range(len(content)):
            content[i] = content[i].replace('\n', ' ')
            if (i == 0) & (content[i][0] == 'A'):
                content[i] = '-'
            elif (i != 0) & (post == content[i][0]):
                N += 1
                content[i-N] += content[i]
                post = content[i][0]
                content[i] = '-'
            else:
                post = content[i][0]
                N = 0

        text = [sentence for sentence in content if (sentence[0] == 'A') or (sentence[0] == 'B')]
        if text[-1][0] == 'B':
            text = text[:-3]
        elif text[-1][0] == 'A':
            text = text[:-2]

        for sentence in text:
            if sentence[0] == 'B':
                dic['Q'].append(sentence)
            elif sentence[0] == 'A':
                dic['A'].append(sentence)
    return pd.DataFrame(dic)

- 01.숙박
- 02.교통
- 03.식당
- 04.레저
- 05.관광

In [3]:
folder_list = ['01.숙박', '02.교통', '03.식당', '04.레저', '05.관광']

In [5]:
DF = pd.DataFrame({'Q':[], 'A':[], 'label':[]})
for folder in folder_list:
    print(folder, folder[1])
    df = create_df(folder)
    df['label'] = int(folder[1])
    DF = pd.concat([DF, df])

01.숙박 1
02.교통 2
03.식당 3
04.레저 4
05.관광 5


In [6]:
DF.Q = DF.Q.str.replace('B.', '')
DF.A = DF.A.str.replace('A.', '')

C:\Users\user\AppData\Local\Temp\ipykernel_12560\3924870456.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  DF.Q = DF.Q.str.replace('B.', '')
C:\Users\user\AppData\Local\Temp\ipykernel_12560\3924870456.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  DF.A = DF.A.str.replace('A.', '')


In [7]:
DF

,Q,A,label
0,네 침구 추가를 해야 할 것 같아서요,네 1세트가 필요하신 건가요,1.0
1,네 맞습니다 추가 요금은 얼마인가요,네 1만 원이시고요 투숙기간 과는 무관합니다,1.0
2,네 알겠습니다 침구문의는 어디다 말씀드려야 할까요,네 프론트에다가 사전에 말씀 부탁 드릴게요,1.0
3,네 예약하려면 어떻게 해야 하나요,네 회원 가입 후 로그인 하셔서 예약하시면 되세요,1.0
4,네 알겠습니다 객실엔 세면 도구가 있나요,네 타월과 욕실 용품은 있지만 개인 용품은 챙겨오셔야 합니다,1.0
...,...,...,...
9436,안녕하세요 여행 계획하는데 궁금한 게 있어 전화드려요,네 고객님 어떤 사항 궁금하실까요,5.0
9437,인터넷에서 해안을 따라 기차를 타는 사진을 봤는데요 부산인 것 같아서요 혹시 알고 ...,네 고객님 설명만으로는 아마도 해운대 블루라인파크 같아요,5.0
9438,해운대 블루라인파크에 대해 알려주실 수 있나요,해운대 미포 청사포 송정에 이르는 철도 시설이에요,5.0
9439,철도 시설이었군요 그럼 어떻게 이용하는 거에요,앞서 말한 세 정거장 중 편하신 정거장에서 탑승하시면 되세요,5.0


In [8]:
DF.to_csv('QA_data.csv')

In [ ]:
chatDF = pd.read_csv('./QA_data.csv', usecols = [1, 2, 3])
chatDF.head()

In [ ]:
with open('stop_word.txt', 'r',encoding = 'utf-8') as f:
    stop_word = f.read().split('\n')
print(stop_word)

In [ ]:
def not_stopword(x):
    sentence = ''
    for word in x.split():
        if word not in stop_word:
            sentence += word + ' '
    return sentence
            

chatDF.Q = chatDF.Q.apply(lambda x: not_stopword(x))
chatDF.A = chatDF.A.apply(lambda x: not_stopword(x))

In [ ]:
chatDF= chatDF[(chatDF.Q != '') & (chatDF.A != '')]
chatDF

In [ ]:
# chatDF.to_csv('chat_data.csv', index=False)